In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pybamm
import pandas as pd
import timeit
start = timeit.default_timer()

In [2]:
model = pybamm.lithium_ion.DFN(
    options={
        "thermal": "lumped",
        "SEI": "interstitial-diffusion limited",
        "SEI on cracks": "true",
        "SEI film resistance": "distributed", 
        "SEI porosity change": "true",
        "particle mechanics": ("swelling and cracking", "swelling only"),
        "loss of active material": "stress-driven",
        "lithium plating": "partially reversible"
    }
)
param = pybamm.ParameterValues("OKane2023")
# Adjust SEI parameters
param.update({"Inner SEI lithium interstitial diffusivity [m2.s-1]": 3e-19})
# Override SEI thickness dependence of lithium plating
param.update({"Dead lithium decay rate [s-1]": 1e-6})
param.update({"Lithium plating kinetic rate constant [m.s-1]": 1e-10})
# Adjust mechanical degradation parameters
param.update({"Negative electrode cracking rate": 1e-22})
param.update({"Negative electrode LAM constant proportional term [s-1]": 1e-9})
param.update({"Positive electrode LAM constant proportional term [s-1]": 1e-9})

In [3]:
# define experiment
V_max = 4.2;        V_min = 2.5; 
Update_Cycles = 10; RPT_Cycles = 1;
charge_time_mins = 60 * 4.86491/5
exp_AGE_text = [(
    f"Charge at 0.3 C for {charge_time_mins} minutes",
    f"Discharge at 1 C until {V_min} V", 
    ),  ]
# step index for ageing
step_AGE_CD =0;   step_AGE_CC =1;   step_AGE_CV =2;

exp_RPT_text = [ (
    f"Charge at 1C until {V_max} V",  
    f"Hold at {V_max}V until C/100",
    f"Discharge at 0.1C until {V_min} V (5 minute period)",  
    "Rest for 1 hours (5 minute period)",  
    f"Charge at 0.1C until {V_max} V (5 minute period)",
    "Rest for 1 hours (5 minute period)",
    f"Discharge at 1C until {V_min} V",  
    f"Hold at {V_min} V until C/100",
    ) ]
# step index for RPT
step_RPT_CD = 2;  step_RPT_RE =3;   step_RPT_CC = 4;  
Experiment_Long   = pybamm.Experiment( exp_AGE_text * Update_Cycles  )  
Experiment_RPT    = pybamm.Experiment( exp_RPT_text * RPT_Cycles     ) 
Experiment_Breakin= pybamm.Experiment( exp_RPT_text * RPT_Cycles     )


In [4]:
var_pts = {
    "x_n": 5,  # negative electrode
    "x_s": 5,  # separator 
    "x_p": 5,  # positive electrode
    "r_n": 60,  # negative particle
    "r_p": 20,  # positive particle
}

In [5]:
sim1 = pybamm.Simulation(
    model, 
    experiment=Experiment_Breakin,
    parameter_values=param,
    solver=pybamm.CasadiSolver(),
    var_pts=var_pts,
)
solution1 = sim1.solve(
    save_at_cycles=1
)

stop = timeit.default_timer()
print('running time: ' + str(stop - start) +'s')

running time: 75.97295170000001s


In [6]:
model_2 = model.set_initial_conditions_from(solution1, inplace=False)
sim2 = pybamm.Simulation(
    model_2, 
    experiment=Experiment_Long,
    parameter_values=param,
    solver=pybamm.CasadiSolver(),
    var_pts=var_pts,
)
solution2 = sim2.solve(
    save_at_cycles=[1,5,10] 
)

In [ ]:
# plot SOH, LAM, LII
# SOH must be defined ourselves? 

In [39]:
label = ["Initial solution","Second solution"] 
var = [
    "Loss of lithium inventory, including electrolyte [%]",
    "LAM_ne [%]",
    "Loss of active material in negative electrode [%]",
    "LAM_pe [%]",
    "Loss of active material in positive electrode [%]",
    'LLI [%]', 
    'Throughput capacity [A.h]',
    "Discharge capacity [A.h]"
]
quick_plot = pybamm.QuickPlot(
    [solution1,solution2], var,label,variable_limits='tight') #     
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=23.175622941368083, step=0.23175622941368082…

In [35]:
solution2['Throughput capacity [A.h]'].entries[0]

14.701643754935972

In [30]:
# need to do this every time next solution is got:
solution2['Throughput capacity [A.h]'].entries += solution1['Throughput capacity [A.h]'].entries[-1]
"Throughput energy [W.h]",

In [ ]:
plt.plot(solution2['Cell temperature [K]'].entries )

In [44]:
solution2['Cell temperature [C]'].entries 

array([[27.81351622, 27.67506053, 27.50454386, ..., 37.00377221,
        37.75469088, 38.0990555 ],
       [27.81351622, 27.67506053, 27.50454386, ..., 37.00377221,
        37.75469088, 38.0990555 ],
       [27.81351622, 27.67506053, 27.50454386, ..., 37.00377221,
        37.75469088, 38.0990555 ],
       ...,
       [27.81351622, 27.67506053, 27.50454386, ..., 37.00377221,
        37.75469088, 38.0990555 ],
       [27.81351622, 27.67506053, 27.50454386, ..., 37.00377221,
        37.75469088, 38.0990555 ],
       [27.81351622, 27.67506053, 27.50454386, ..., 37.00377221,
        37.75469088, 38.0990555 ]])